In [746]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

sys.path.append('fbsource/fbcode/scripts/psarlin/')
from maploc.utils.geometry import Pose, Camera
from maploc.utils.io import read_image
from maploc.utils.viz_2d import plot_images, plot_keypoints, add_text, save_plot

In [751]:
root = Path('buckets/llp_datasets/holicity/')
with open(root / 'split/filelist.txt') as fid:
    prefixes = [l.strip('\n') for l in fid]
prefix = prefixes[1500]

image = read_image(root / f'image/{prefix}_imag.jpg')
cam_data = np.load(root / f'camr/{prefix}_camr.npz')
cam = Camera.from_K(K, image.shape)

pano_prefix = prefix[:-10]
pano = read_image('./data/panorama/'+pano_prefix+'.jpg')

plot_images([image, pano])


In [748]:
names = !ls $root/image/$pano_prefix*
plot_images([read_image(n) for n in names], titles=[n.split('_')[-3:-1] for n in names])

In [749]:
import json
with open(f'data/panorama-camera/{pano_prefix}_LD_camr.json', 'r') as fp:
    pano_cam = json.load(fp)

In [798]:
for prefix in np.random.RandomState(2).choice(prefixes, 10):
    image = read_image(root / f'image/{prefix}_imag.jpg')
    cam_data = np.load(root / f'camr/{prefix}_camr.npz')
    pano_prefix = prefix[:-10]
    pano = read_image('./data/panorama/'+pano_prefix+'.jpg')

    pitch = np.random.randint(-10, 11)
    # render = render_perspective(pano, cam_data, cam_data["yaw"], cam_data["pitch"], (512, 512))
    render = render_perspective(pano, cam_data, cam_data["yaw"], pitch, (512, 512))
    plot_images([image, render], titles=[cam_data["pitch"], pitch], dpi=50)

In [864]:
from scipy.spatial.transform import Rotation

def build_perspective_4x4mat(loc, yaw, pitch):
    """Computes 4x4 world-to-camera transformation matrix"""
    yaw = -yaw * np.pi / 180 + np.pi / 2
    pitch = pitch * np.pi / 180
    return lookat(
        loc,
        [np.cos(pitch) * np.cos(yaw), np.cos(pitch) * np.sin(yaw), np.sin(pitch)],
        [0, 0, 1],
    )

def lookat(position, forward, up=[0, 1, 0]):
    """Computes 4x4 transformation matrix to put camera looking at look point."""
    c = np.asarray(position).astype(float)
    w = -np.asarray(forward).astype(float)
    u = np.cross(up, w)
    v = np.cross(w, u)
    u /= np.linalg.norm(u)
    v /= np.linalg.norm(v)
    w /= np.linalg.norm(w)
    return np.r_[u, u.dot(-c), v, v.dot(-c), w, w.dot(-c), 0, 0, 0, 1].reshape(4, 4)

def proj_pano(d, image_size):
    # assume that Z is up, Y is forward
    d = d.T  # Nx3 to 3xN
    pitch = np.arctan2(d[2], np.linalg.norm(d[:2], axis=0))
    yaw = np.arctan2(d[0], d[1])
    x = (yaw + np.pi) / (np.pi * 2)
    y = 1 - (pitch + np.pi / 2) / np.pi
    xy_norm = np.stack([x, y], -1)
    xy = xy_norm * np.array(image_size)
    return xy, xy_norm

def render_perspective(pano, cam_data, yaw, pitch, image_size):
    pano_yaw = np.deg2rad(cam_data['pano_yaw'])
    tilt_yaw = cam_data['tilt_yaw']
    tilt_pitch = np.deg2rad(cam_data['tilt_pitch'])
    
    # perspective -> CV: (x,y,z) -> (x,-y,-z)
    R_cam2cv = np.diag([1, -1, -1])
    R_w2cam = R_cam2cv @ build_perspective_4x4mat(np.zeros(3), yaw, pitch)[:3, :3]
    axis = np.cross([np.cos(tilt_yaw), np.sin(tilt_yaw), 0], [0, 0, 1])
    # Z is up, Y is forward, X is right
    R_w2pano = (Rotation.from_rotvec(-axis*tilt_pitch) * Rotation.from_euler('z', -pano_yaw)).as_matrix()
    R_w2pano = (Rotation.from_euler('z', -pano_yaw) * Rotation.from_rotvec(-axis*tilt_pitch)).as_matrix()
    R_cam2pano = R_w2pano @ R_w2cam.T

    grid = np.stack(np.indices(image_size[::-1]), -1)[..., ::-1]
    w, h = image_size
    fx, fy, cx, cy = w/2, h/2, w/2, h/2. # 90 FOV
    p2d_norm = (grid.reshape(-1, 2) - np.array([cx, cy])) / np.array([fx, fy])

    bearing_cam = to_homogeneous(p2d_norm)
    bearing_cam = bearing_cam / np.linalg.norm(bearing_cam, axis=-1, keepdims=True)
    bearing_pano = bearing_cam @ R_cam2pano.T
    # Z is up, Y is forward, X is right

    pano_size = pano.shape[:2][::-1]
    p2d_pano, p2d_pano_norm = proj_pano(bearing_pano, pano_size)
    p2d_pano_grid = p2d_pano.reshape(grid.shape).astype(np.float32)

    image = cv2.remap(pano, *p2d_pano_grid.transpose(2, 0, 1), cv2.INTER_LINEAR, borderMode=cv2.BORDER_WRAP)
    return image

%time render = render_perspective(pano, cam_data, cam_data["yaw"], cam_data["pitch"], (512, 512))

# gnorm = grid.reshape(-1, 2)/np.array(image.shape[:2][::-1])
# gridcolor = color2d(*gnorm.T)[:, :3].reshape(grid.shape[:2]+(3,))
# plot_images([pano], dpi=50)
# plt.scatter(*p2d_pano.T, c=gridcolor.reshape(-1, 3), s=4)

plot_images([image, render])
diff = np.abs(image/255. - render/255.).mean(-1)
plot_images([(image/255.+render/255.)/2, diff])
print(diff.mean())

In [ ]:
from collections import defaultdict
from multiprocessing import Pool
import functools
import shutil
import cv2

root = Path('buckets/llp_datasets/holicity/')
with open(root / 'split/filelist.txt') as fid:
    prefixes = [l.strip('\n') for l in fid]

out_dir = Path('./data/holicity_renders')
camera_dir = out_dir / "camr"
image_dir = out_dir / "image"
if camera_dir.exists():
    shutil.rmtree(camera_dir)
if image_dir.exists():
    shutil.rmtree(image_dir)

pano2prefixes = defaultdict(list)
for prefix in prefixes:
    pano = '_'.join(prefix.split('_')[:-3])
    pano2prefixes[pano].append(prefix)
pano2prefixes = dict(pano2prefixes)
panos = list(pano2prefixes)
print(f"Doing {len(panos)} panoramas")

def do_pano(idx, pano2prefixes, root, camera_dir, image_dir):
    pano_name, prefixes = pano2prefixes[idx]

    pano = read_image('./data/panorama/'+pano_name+'.jpg')
    prefixes_new = []
    for prefix in prefixes:
        image = read_image(root / f'image/{prefix}_imag.jpg')
        with np.load(root / f'camr/{prefix}_camr.npz') as cam_data:
            cam_data = dict(cam_data)

        pitch = np.random.randint(-15, 16)
        prefix_new = '_'.join(prefix.split('_')[:-1]+[f'{pitch:+03d}'])
        cam_data["pitch"] = pitch
        render = render_perspective(pano, cam_data, cam_data["yaw"], pitch, (512, 512))

        (camera_dir / prefix_new).parent.mkdir(exist_ok=True, parents=True)
        (image_dir / prefix_new).parent.mkdir(exist_ok=True, parents=True)
        np.savez(camera_dir / f'{prefix_new}_camr.npz', **cam_data)
        cv2.imwrite(str(image_dir / f'{prefix_new}_imag.jpg'), render[:, :, ::-1])
        prefixes_new.append(prefix_new)
    print("Done pano", idx)
    return prefixes_new

fn = functools.partial(
    do_pano, pano2prefixes=list(pano2prefixes.items()), root=root, camera_dir=camera_dir, image_dir=image_dir)
with Pool(10) as p:
    prefixes_new = p.map(fn, range(len(panos)))

prefixes_new = [p for pp in prefixes_new for p in pp]
with open(out_dir / "split/filelist.txt", 'w') as fp:
    fp.write("\n".join(prefixes_new))

In [868]:
with open(out_dir / 'split/filelist.txt') as fid:
    prefixes = [l.strip('\n') for l in fid]
for prefix in tqdm(prefixes):
    with np.load(out_dir / f'camr/{prefix}_camr.npz') as cam_data:
        cam_data = dict(cam_data)
    cam_data["R"] = build_perspective_4x4mat(cam_data["loc"], cam_data["yaw"], cam_data["pitch"])
    np.savez(out_dir / f'camr/{prefix}_camr.npz', **cam_data)

In [436]:
from matplotlib import cm
def color2d(x, y):
    return cm.hsv(x)*y[..., None] + (1-y[..., None])

gnorm = grid.reshape(-1, 2)/np.array(image.shape[:2][::-1])
gridcolor = color2d(*gnorm.T)[:, :3].reshape(grid.shape[:2]+(3,))
plot_images([gridcolor])